# Questions List (Answers follow after)

[Source](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/02-open-source/homework.md)

In this homework, we'll experiment more with Ollama

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Ollama with Docker

Let's run ollama with Docker. We will need to execute the 
same command as in the lectures:

```bash
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

What's the version of ollama client? 

To find out, enter the container and execute `ollama` with the `-v` flag.


## Q2. Downloading an LLM 

We will donwload a smaller LLM - gemma:2b. 

Again let's enter the container and pull the model:

```bash
ollama pull gemma:2b
```

In docker, it saved the results into `/root/.ollama`

We're interested in the metadata about this model. You can find
it in `models/manifests/registry.ollama.ai/library`

What's the content of the file related to gemma?

## Q3. Running the LLM

Test the following prompt: "10 * 10". What's the answer?

## Q4. Donwloading the weights 

We don't want to pull the weights every time we run
a docker container. Let's do it once and have them available
every time we start a container.

First, we will need to change how we run the container.

Instead of mapping the `/root/.ollama` folder to a named volume,
let's map it to a local directory:

```bash
mkdir ollama_files

docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Now pull the model:

```bash
docker exec -it ollama ollama pull gemma:2b 
```

What's the size of the `ollama_files/models` folder? 

* 0.6G
* 1.2G
* 1.7G
* 2.2G

Hint: on linux, you can use `du -h` for that.

## Q5. Adding the weights 

Let's now stop the container and add the weights 
to a new image

For that, let's create a `Dockerfile`:

```dockerfile
FROM ollama/ollama

COPY ...
```

What do you put after `COPY`?

## Q6. Serving it 

Let's build it:

```bash
docker build -t ollama-gemma2b .
```

And run it:

```bash
docker run -it --rm -p 11434:11434 ollama-gemma2b
```

We can connect to it using the OpenAI client

Let's test it with the following prompt:

```python
prompt = "What's the formula for energy?"
```

Also, to make results reproducible, set the `temperature` parameter to 0:

```bash
response = client.chat.completions.create(
    #...
    temperature=0.0
)
```

How many completion tokens did you get in response?

* 304
* 604
* 904
* 1204

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw2
* It's possible that your answers won't match exactly. If it's the case, select the closest one.

---

# Answers

## Q1
ollama version is 0.1.48

## Q2

What I did:
1. `docker start ollama`

2. `docker exec -it ollama /bin/bash`

3. `cd /root/.ollama/models/manifests/registry.ollama.ai/library/gemma`

    Existing is `2b`.

4. `cat 2b`
   
The content:

{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}


## Q3
`ollama run gemma:2b`

`Answer:`

Sure. Here's the answer to your question:

10 * 10 = 100.


## Q4: Downloading the weights



Size of the `ollama_files/models` folder: 

1,678,463,660 bytes (1.68 GB on disk) for 12 items



## Q5: Adding the weights

COPY models/blobs /root/.ollama/models/blobs

## Q6: Serving it

In [16]:
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv('/app/.env')
api_key=os.getenv('OPENAI_API_KEY')

client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',
)

# openai.api_base = 'http://localhost:11434/v1'
# openai.api_key = os.getenv('OPENAI_API_KEY')

In [39]:
def get_completion(prompt):
    response = client.chat.completions.create(
        model="gemma:2b",
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=0.0
    )
    response_text = response.choices[0].message.content
    completion_tokens = response.usage.completion_tokens
    return response_text, completion_tokens

In [41]:
prompt = "What's the formula for energy?"

response_text, completion_tokens = get_completion(prompt)

print("Response:\n", response_text)

Response:
 Sure, here's the formula for energy:

**E = m * c^2**

Where:

* **E** is the energy in joules (J)
* **m** is the mass in kilograms (kg)
* **c** is the speed of light in a vacuum squared (m/s^2)

This formula tells us that energy is a measure of the amount of work that can be done or the amount of energy stored in an object.


In [42]:
### Counting the number of tokens
print("Number of completion tokens used:\n", completion_tokens)

Number of completion tokens used:
 101
